In [1]:
from tqdm import tqdm
import pandas as pd
import fasttext
import numpy as np

In [2]:
model = fasttext.load_model("/home/burak/Desktop/cc.en.300.bin")

In [3]:
df = pd.read_csv("processed.csv",encoding="utf-8",lineterminator="\n")

In [4]:
df

,content,label
0,rt escaping the sandbox - microsoft office in...,CYBER
1,office 365 biz users targeted in novel phish s...,CYBER
2,adobe flash player flaws remain the most used ...,CYBER
3,bug in mingw-w64 windows app development envir...,CYBER
4,threatmeter: microsoft windows 7 sp1 nfo file ...,CYBER
...,...,...
81165,so grateful for my health what are you gratef...,NOT-CYBER
81166,my prayers are with paris.... i love you..,NOT-CYBER
81167,life couldn't be better right now.,NOT-CYBER
81168,first monday back in action. i'd say 21.6 mile...,NOT-CYBER


In [6]:
sent_vectors = []
for item in tqdm(df.iterrows()):
    sent_vectors.append(model.get_sentence_vector(item[1].content))
    
sent_vectors = np.array(sent_vectors)

81170it [00:13, 6140.20it/s]


In [7]:
labels_encoded = []
for item in tqdm(df.iterrows()):
    if item[1].label == "CYBER":
        labels_encoded.append(1)
    else:
        labels_encoded.append(0)
labels_encoded=np.array(labels_encoded)

81170it [00:07, 11256.16it/s]


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(sent_vectors,labels_encoded,test_size=0.15)

In [9]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [10]:
clf.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [11]:
score = clf.score(X_test, Y_test)
score

0.9729796320630749

In [12]:
import pickle
filename = 'svc.model'
pickle.dump(clf, open(filename, 'wb'))

# # some time later...

# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [16]:
clf.predict([model.get_sentence_vector("today i've buy an asus laptop")])

array([1])